[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/quick-tour/hello-pinecone.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/docs/quick-tour/hello-pinecone.ipynb)

# Hello, Pinecone!

This notebook will walk through the steps to get a simple Pinecone index up and running.


## Prerequisites

Install dependencies.

In [1]:
!pip install -qU \
  pinecone-client==3.0.0 \
  pandas==2.0.3

Set up Pinecone. Get your Pinecone API key [here](https://www.pinecone.io/start).

Before getting started, decide whether to use serverless or pod-based index.

In [ ]:
import os

use_serverless = os.environ.get("USE_SERVERLESS", "False").lower() == "true"

## Creating an Index

Now the data is ready, we can set up our index to store it.

We begin by initializing our connection to Pinecone. To do this we need a [free API key](https://app.pinecone.io).

In [ ]:
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pc.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
environment = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

# configure client
pc = Pinecone(api_key=api_key)

Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [ ]:
from pinecone import ServerlessSpec, PodSpec

if use_serverless:
    spec = ServerlessSpec(cloud='aws', region='us-west-2')
else:
    spec = PodSpec(environment=environment)

## Pinecone quickstart

With Pinecone you can create a vector index where you can store and search through your vectors.

In [3]:
# Giving our index a name
index_name = "hello-pinecone"

In [4]:
# Delete the index, if an index of the same name already exists
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

Creating a Pinecone Index.

In [5]:
import time

dimensions = 3
pc.create_index(
    name=index_name, 
    dimension=dimensions, 
    metric="cosine",
    spec=spec
)

# wait for index to be ready before connecting
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

In [6]:
index = pc.Index(index_name)

We have the index ready. Now we will create some simple vectors that will serve as our examples.

In [7]:
import pandas as pd

df = pd.DataFrame(
    data={
        "id": ["A", "B"],
        "vector": [[1., 1., 1.], [1., 2., 3.]]
    })
df

,id,vector
0,A,"[1.0, 1.0, 1.0]"
1,B,"[1.0, 2.0, 3.0]"


We perform upsert operations in our index. This call will insert a new vector in the index or update the vector if the id was already present.

In [8]:
index.upsert(vectors=zip(df.id, df.vector))  # insert vectors

{'upserted_count': 2}

In [9]:
index.describe_index_stats()

{'dimension': 3,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [10]:
index.query(
    vector=[2., 2., 2.],
    top_k=5,
    include_values=True) # returns top_k matches

{'matches': [], 'namespace': ''}

## Delete the Index
Delete the index once you are sure that you do not want to use it anymore. Once the index is deleted, you cannot use it again.

In [11]:
pc.delete_index(index_name)